<a href="https://colab.research.google.com/github/AlexeyTri/SemMed_fall25/blob/main/Seminar1/SemMed1_fall25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>